In [127]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber 
from nipype.interfaces.utility import IdentityInterface, Function    
from nipype.pipeline.engine import Node, Workflow, JoinNode, MapNode
from nipype.interfaces import fsl
from nipype.interfaces import brainsuite
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from shutil import copyfile
from pandas import Series, read_csv, to_numeric
from glob import glob
from os.path import abspath, expanduser, join
from os import chdir, remove, getcwd
from nipype import config, logging
from datetime import date
today = str(date.today())
config.enable_debug_mode()

In [225]:
#Set variables
subject_list = ['001-prac']

user = expanduser('~')
home = join(user,'Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data')
if user == '/Users/lucindasisk':
    workflow_dir = join(home, 'workflow_LS')
    data_dir = join(home, 'DataDir_LS')
elif user == '/Users/myelin':
    workflow_dir = join(home, 'workflow')
    data_dir = join(home, 'DataDir')

i = read_csv(join(home, 'index.txt'), sep = ' ', header=None, dtype='Int64')
il = i.loc[0]
index = il.tolist()

In [241]:
#Setup Datasink, Infosource, Selectfiles

datasink = Node(DataSink(base_directory = data_dir),
                   name='datasink')

#Set infosource iterables
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

#SelectFiles
template = dict(t1 = join(home,'{subject_id}/125-skullstripped_anat.nii.gz'),
                pe1= join(home, '{subject_id}/dti_pe1.nii.gz'),
                pe0= join(home, '{subject_id}/dti_pe0.nii.gz'),
                bvalpe1 = join(home,'{subject_id}/dti_pe1.bval'),
                bvalpe0 = join(home,'{subject_id}/dti_pe0.bval'),
                bvecpe1 = join(home,'{subject_id}/dti_pe1.bvec'),
                bvecpe0 = join(home,'{subject_id}/dti_pe0.bvec'),
                index = join(home, 'index.txt'),
                aps = join(home, 'acqparams.txt'),
                tryxfm = join(home, 'transform.txt'))

sf = Node(SelectFiles(template, 
                      base_directory = home),
          name = 'sf')
                

In [242]:
#Create ROIs from b0 volumes for pe1 and pe0, then merge
def create_merged_files(pe1, pe0):
    from nipype.interfaces import fsl
    from os.path import abspath
    fslroi1 = fsl.ExtractROI(in_file = pe1, t_min = 0, t_size = 1, roi_file = 'pe1_1vol.nii.gz').run()  
    fslroi2 = fsl.ExtractROI(in_file = pe0, t_min = 0, t_size = 1, roi_file = 'pe0_1vol.nii.gz').run()  
    merge = fsl.Merge(in_files = ['pe1_1vol.nii.gz', 'pe0_1vol.nii.gz'], 
                      dimension = 't', merged_file = 'pe1_pe0_merged.nii.gz').run()
    merged_file = abspath('pe1_pe0_merged.nii.gz')
    return merged_file

create_merge = Node(Function(input_names=['pe1', 'pe0'],
                             output_names=['merged_file'],
                             function = create_merged_files),
                    name='create_merge')

In [243]:
#Run topup on merged files from pe1 and pe0
topup = Node(fsl.TOPUP(config = 'b02b0.cnf'),
            name = 'topup',
            out_corrected = 'pe1_pe0_topup.nii.gz')

#apply topup from merged file to rest of pe0 scan
apptop = Node(fsl.ApplyTOPUP(out_corrected = 'pe0_applied_topup.nii.gz',
                            method='jac',
                            in_index=[2]),
             name = 'apptop')

#Skullstrip the T1w image
stripT1 = Node(fsl.BET(mask = True,
                      out_file = 'T1_stripped_anat.nii.gz'),
              name = 'stripT1')

#Reorient DTI data to standard orientation
reorient = Node(fsl.Reorient2Std(out_file = 'pe0_applied_topup_reoriented.nii.gz',
                              output_type = 'NIFTI_GZ'),
                    name = 'reorient')

#Flirt node to resample DTI data to same dimensions as T1 (registration)
resample = Node(fsl.FLIRT(out_file = 'pe0_applied_topup_reoriented_resampled.nii.gz',
                          output_type = 'NIFTI_GZ',
                          apply_xfm=True),
                    name = 'resample')

#FSL Eddy correction to remove eddy current distortion
eddy = Node(fsl.Eddy(),
           name = 'eddy')


In [ ]:
preproc_flow = Workflow(name = 'preproc_flow')
preproc_flow.connect([(infosource, sf, [('subject_id','subject_id')]),
                      (sf, create_merge, [('pe1', 'pe1'),
                                          ('pe0','pe0')]),
                      (create_merge, datasink, [('merged_file', 'test')]),
                      (create_merge, topup, [('merged_file', 'in_file')]),
                      (sf, topup, [('aps','encoding_file')]),
                      (topup, datasink,[('out_corrected', 'test.@par')]),
                      (topup, apptop, [('out_fieldcoef', 'in_topup_fieldcoef'),
                                       ('out_movpar', 'in_topup_movpar')]),
                      (sf, apptop, [('pe0', 'in_files'),
                                    ('aps', 'encoding_file')]),
                      (apptop, datasink, [('out_corrected', 'test.@par.@par')]),
                      (sf, stripT1, [('t1', 'in_file')]),
                      (stripT1, datasink, [('out_file', 'test.@par.@par.@par'),
                                           ('mask_file', 'test.@par.@par.@par.@par')]),
                      (apptop, resample, [('out_corrected','in_file')]),
                      (sf, resample, [('t1','reference'),
                                       ('tryxfm', 'in_matrix_file')]),
                      
                      (resample, reorient, [('out_file', 'in_file')]),
                      (reorient, datasink, [('out_file', 'test.@par.@par.@par.@par.@par')]),
                      (sf, eddy, [('index', 'in_index'),
                                 ('bvecpe0', 'in_bvec'),
                                 ('bvalpe0', 'in_bval'),
                                 ('aps', 'in_acqp')]),
                      (stripT1, eddy, [('mask_file', 'in_mask')]),
                      (reorient, eddy, [('out_file','in_file')]),
                      (eddy, datasink,[('out_corrected','eddy_corrected')])
                     ])
preproc_flow.base_dir = workflow_dir
preproc_flow.write_graph(graph2use = 'flat')
preproc = preproc_flow.run('MultiProc', plugin_args={'n_procs': 2})

190404-18:38:56,193 nipype.workflow DEBUG:
	 (preproc_flow.infosource, preproc_flow.sf): No edge data
190404-18:38:56,194 nipype.workflow DEBUG:
	 (preproc_flow.infosource, preproc_flow.sf): new edge data: {'connect': [('subject_id', 'subject_id')]}
190404-18:38:56,195 nipype.workflow DEBUG:
	 (preproc_flow.sf, preproc_flow.create_merge): No edge data
190404-18:38:56,196 nipype.workflow DEBUG:
	 (preproc_flow.sf, preproc_flow.create_merge): new edge data: {'connect': [('pe1', 'pe1'), ('pe0', 'pe0')]}
190404-18:38:56,197 nipype.workflow DEBUG:
	 (preproc_flow.create_merge, preproc_flow.datasink): No edge data
190404-18:38:56,197 nipype.workflow DEBUG:
	 (preproc_flow.create_merge, preproc_flow.datasink): new edge data: {'connect': [('merged_file', 'test')]}
190404-18:38:56,198 nipype.workflow DEBUG:
	 (preproc_flow.create_merge, preproc_flow.topup): No edge data
190404-18:38:56,201 nipype.workflow DEBUG:
	 (preproc_flow.create_merge, preproc_flow.topup): new edge data: {'connect': [('me

190404-18:38:57,429 nipype.workflow DEBUG:
	 [Node] infosource - setting input subject_id = 001-prac
190404-18:38:57,429 nipype.workflow DEBUG:
	 Parameterization: paramstr=_subject_id_001-prac
190404-18:38:57,430 nipype.workflow DEBUG:
	 PE: expanding iterables ... done
190404-18:38:57,432 nipype.workflow DEBUG:
	 [Node] sf - setting input subject_id = 001-prac
190404-18:38:57,433 nipype.workflow DEBUG:
	 Removed the identity node preproc_flow.infosource from the graph.
190404-18:38:57,437 nipype.workflow DEBUG:
	 Performing depth first search
190404-18:38:57,440 nipype.workflow INFO:
	 Running in parallel.
190404-18:38:57,443 nipype.workflow DEBUG:
	 Progress: 9 jobs, 0/0/1 (done/running/ready), 0/9 (pending_tasks/waiting).
190404-18:38:57,443 nipype.workflow DEBUG:
	 Tasks currently running: 0. Pending: 0.
190404-18:38:57,445 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 1 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 2/2.
190404-18:38:57,865 nipype.workflow

190404-18:38:59,921 nipype.utils DEBUG:
	 Loading pkl: /Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/workflow/preproc_flow/_subject_id_001-prac/sf/result_sf.pklz
190404-18:38:59,922 nipype.workflow DEBUG:
	 output: pe1
190404-18:38:59,923 nipype.workflow DEBUG:
	 [Node] create_merge - setting input pe1 = /Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/001-prac/dti_pe1.nii.gz
190404-18:38:59,924 nipype.workflow DEBUG:
	 input: pe0
190404-18:38:59,924 nipype.workflow DEBUG:
	 results file: /Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/workflow/preproc_flow/_subject_id_001-prac/sf/result_sf.pklz
190404-18:38:59,925 nipype.utils DEBUG:
	 Loading pkl: /Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/workflow/preproc_flow/_subject_id_001-prac/sf/result_sf.pklz
190404-18:38:59,927 nipype.workflow DEBUG:
	 output: pe0
190404-18:38:59,928 nipype.workflow DEBUG:
	 [Node] create_merge - setting input pe0 = /Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_

190404-18:39:03,857 nipype.workflow DEBUG:
	 input: encoding_file
190404-18:39:03,858 nipype.workflow DEBUG:
	 results file: /Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/workflow/preproc_flow/_subject_id_001-prac/sf/result_sf.pklz
190404-18:39:03,858 nipype.utils DEBUG:
	 Loading pkl: /Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/workflow/preproc_flow/_subject_id_001-prac/sf/result_sf.pklz
190404-18:39:03,860 nipype.workflow DEBUG:
	 output: aps
190404-18:39:03,860 nipype.workflow DEBUG:
	 [Node] apptop - setting input encoding_file = /Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/acqparams.txt
190404-18:39:03,863 nipype.workflow DEBUG:
	 [Node] Hashes: [('encoding_file', ('/Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/acqparams.txt', '7411cdbb2460ae66c3413f466d4e1068')), ('in_files', [('/Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/001-prac/dti_pe0.nii.gz', '0620b921ab7f61df684a27149fabe4e4')]), ('in_index', [2]), ('in_topup_field

	 out_matrix_file_pe0_applied_topup_flirt.mat
190404-18:39:05,952 nipype.interface DEBUG:
	 reference_/Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/001-prac/125-skullstripped_anat.nii.gz
190404-18:39:05,957 nipype.workflow INFO:
	 [Node] Running "resample2" ("nipype.interfaces.fsl.preprocess.FLIRT"), a CommandLine Interface with command:
flirt -in /Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/workflow/preproc_flow/_subject_id_001-prac/apptop/pe0_applied_topup.nii.gz -ref /Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/001-prac/125-skullstripped_anat.nii.gz -out pe0_applied_topup_reoriented_resampled.nii.gz -omat pe0_applied_topup_flirt.mat -applyxfm -init /Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/transform.txt
190404-18:39:05,963 nipype.interface DEBUG:
	 apply_xfm_True
190404-18:39:05,969 nipype.interface DEBUG:
	 in_file_/Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/workflow/preproc_flow/_subject_id_001-prac/apptop/pe0_applied_

	 [Node] Hashes: [('in_file', ('/Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/workflow/preproc_flow/_subject_id_001-prac/resample2/pe0_applied_topup_reoriented_resampled.nii.gz', 'e6984578b230f04274238479105f9d42')), ('out_file', 'pe0_applied_topup_reoriented.nii.gz'), ('output_type', 'NIFTI_GZ')], 2aafd1e82cb2106441199148263fbec6, /Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/workflow/preproc_flow/_subject_id_001-prac/reorient/_0x2aafd1e82cb2106441199148263fbec6.json, ['/Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/workflow/preproc_flow/_subject_id_001-prac/reorient/_0xb3e7e76c8007a2ca5bb8812810d4ba76.json']
190404-18:42:12,26 nipype.workflow INFO:
	 [Node] Outdated cache found for "preproc_flow.reorient".
190404-18:42:12,31 nipype.workflow DEBUG:
	 [Node] Old/new hashes = b3e7e76c8007a2ca5bb8812810d4ba76/2aafd1e82cb2106441199148263fbec6
190404-18:42:12,35 nipype.workflow DEBUG:
	           Some dictionary entries had differing values:
            * in

190404-18:42:34,5 nipype.utils DEBUG:
	 Loading pkl: /Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/workflow/preproc_flow/_subject_id_001-prac/sf/result_sf.pklz
190404-18:42:34,6 nipype.workflow DEBUG:
	 output: aps
190404-18:42:34,7 nipype.workflow DEBUG:
	 [Node] eddy - setting input in_acqp = /Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/acqparams.txt
190404-18:42:34,8 nipype.workflow DEBUG:
	 input: in_mask
190404-18:42:34,8 nipype.workflow DEBUG:
	 results file: /Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/workflow/preproc_flow/_subject_id_001-prac/stripT1/result_stripT1.pklz
190404-18:42:34,9 nipype.utils DEBUG:
	 Loading pkl: /Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_data/workflow/preproc_flow/_subject_id_001-prac/stripT1/result_stripT1.pklz
190404-18:42:34,32 nipype.workflow DEBUG:
	 output: mask_file
190404-18:42:34,33 nipype.workflow DEBUG:
	 [Node] eddy - setting input in_mask = /Users/myelin/Dropbox/SNAPLab/Data/ELS_AFQ_prac/dwi_da

190404-18:42:35,619 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 0 jobs ready. Free memory (GB): 28.60/28.80, Free processors: 1/2.
                     Currently running:
                       * preproc_flow.eddy
